In [2]:
# Example texts
text1 = """カナダ人の留学生です。日本の大学で勉強しています。私の悩みは日本語です。今、日本人のホストファミリーと住んでいます。家族は親切
ですが、みんなは私と英語を話したがっています。だから、私は「英語を話さなきゃ」と思って、英語を話します。学校に日本人の友達がたくさんいますが、
みんなの英語は私の日本語より上手です。だから、たいてい英語を使います。買い物の時も「すみません。あのう、これください」と日本語で言いますが、
お店の人は「ツーハンドレッドエンね。サンキュー！」と英語で言います。もう六か月も日本にいますが、ぜんぜん日本語が上手になりません。どうしたら
いいでしょうか。"""
text2 ="""武士（サムライ）はもともと9世紀頃の宮廷社会で身分の高い人々に仕えて身辺警護をする人々の呼び名でした。
侍の語源、「さぶらう」、には、人に仕える、という意味があります。その後貴族社会が力を失い、武士が支配する社会になると、戦争で戦う者を侍
と呼ぶようになりました。15世紀半ばから江戸幕府が成立した1603年まで続いた戦国時代においては、出身身分に関わらず戦場での働きによって侍になる
ことができました。江戸時代になって、士農工商という身分制度が設定されました。この身分制度の中で、武士（サムライ）は最高位に置かれ、日本を
統治する階級でした。
江戸時代以降、実際に戦場で戦うことはなくなりましたが、現在武士道として知られる独自の理念、思想が武士の子孫たちに受け継がれていきました。
礼儀や心得などはそれぞれの武家で解釈が異なる場合もありますが、一般的に、「自らの行動や責任に命をかける」、「一命を捨ててでも殿に仕える」、
「失敗をした際には自分の命を捨てる覚悟を持つ」などというものでした。共通するのは、「命をかける」という点で、このような思想から「切腹」
という慣習がうまれました。切腹とは、失敗した際には、自らの腹を斬り、命を捨てることでけじめを付けるというものです。
1868年に将軍徳川慶喜が明治天皇に大政奉還して、武士が日本を統治する時代は終わりました（明治維新）。士農工商の身分制度は廃止され、
身分は華族と平民にわけられました。華族になった一部の上層武家以外は、平民として自分たちで生活費を稼いで生きていく必要に迫られました。
武士という身分はなくなっても、多くの元武士たちは、武士としての精神性を大切にしていました。
そのような精神性については、海外において、新渡戸稲造が1900年に米国でというタイトルの本を英語で出版し、
その日本語版が日本でも出版されて版を重ねたことから武士道という呼び方で知られるようになりました。"""

In [3]:
# imports
import fugashi
from fugashi import Tagger
from sudachipy import tokenizer
from sudachipy import dictionary
import pykakasi
kks = pykakasi.kakasi()

In [4]:
# regex
hiragana_full = r'[ぁ-ゟ]'
katakana_full = r'[゠-ヿ]'
kanji = r'[㐀-䶵一-鿋豈-頻]'
radicals = r'[⺀-⿕]'
katakana_half_width = r'[｟-ﾟ]'
alphanum_full = r'[！-～]'
symbols_punct = r'[、-〿]'
misc_symbols = r'[ㇰ-ㇿ㈠-㉃㊀-㋾㌀-㍿]'
ascii_char = r'[ -~]'
particles = ["は", "に", "を", "も", "へ", "で", "が", "て", "から", "まで", "と", "や", "の", "ね",
            "よ", "て", "た", "ます", "より", "たら", "でしょう", "か", "ませ", "ん", "し", "い", "なきゃ", "たがっ", "う", "でし"
            "まし", "まし", "られ", "れ", "れる", "でし", "なり"]

In [5]:
# Phrases
phrases = ["ようになりました", "このような", "て", "くる", "て" "いる"]

In [6]:
#Tokenizing
tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C
tokens = [m.surface() for m in tokenizer_obj.tokenize(text2, mode)] #<<< change text name here

In [12]:
class Word:
    def __init__(self, word, pos, dictionary_form, hira, kana, roma):
        """ Word object with attributes Part of speech, Dictionary form, Hiragana
            Katakana and Romaji"""
        self._word = word
        self._next = None
        self._pos = pos
        self._dictionary_form = dictionary_form
        self._hira = hira
        self._kana = kana
        self._roma = roma

    def get_word(self):
        return self._word
    
    def get_dictionary_form(self):
        return self._dictionary_form

    def get_pos(self):
        return(self._pos)
        
    def get_hira(self):
        return(self._hira)
        
    def get_kana(self):
        return(self._kana)
        
    def get_romaji(self):
        return(self._roma)

def create_text(input_words_list): #creating text from a list of word objects
    text_output = ""
    i = 0
    text_output += input_words_list[i].get_word()
    no_space = False
    for i in range(len(input_words_list)-1):
        i+=1
        current = input_words_list[i]
        if current.get_pos().startswith("助詞"): #particles
            text_output+= current.get_word()
            #print("particle ", current.get_word())
        elif current.get_pos().startswith("名詞"): #nouns
            text_output+= " "
            text_output+= current.get_word()
            #print("noun ", current.get_word())
            if input_words_list[i+1].get_dictionary_form() and input_words_list[i+1].get_dictionary_form().startswith("為る"):
                no_space = True #detecting suru verbs
                #print("suru verb p1 ", current.get_word())
        elif current.get_dictionary_form() and current.get_dictionary_form().startswith("為る") and no_space:
            text_output+= current.get_word() #detecting suru verbs
            #print("suru verb p2", current.get_word())
            no_space = False
        elif current.get_word() in particles: #particles that appear in the middle of words
            #print("particle ", current.get_word())
            text_output+= current.get_word() 
        elif current.get_word() in symbols_punct or current.get_word() in alphanum_full + "。": #detecting other symbols
            text_output+= current.get_word()  
            #print("alphanum ", current.get_word())
        else:
                text_output+= " "
                text_output+= current.get_word()
                #print("other ", current.get_word(), " ", i)
            
    return(text_output) 

def populate_nodes(tokens): # creates a list of word objects
    filtered_tokens = [i for i in tokens if i not in ["\n", None]] #filter out the endline characters
    tagger = Tagger('-Owakati')
    nodes_list = []
    for token in filtered_tokens:
        tagger.parse(token) #parsing with fugashi
        parsed_word = tagger(token)[0]
        POS = (parsed_word.pos.split(",")[0]) #extracting part of speech
        dict_form = parsed_word.feature.lemma #extracting dictionary form
        converted_word = kks.convert(token) #converting into a kks object to extract other attributes
        item = converted_word[0]
        orig, dict_form, hiragana, katakana, romaji = (token, dict_form, item['hira'], item['kana'], item['kunrei'])
        word = Word(orig, POS, dict_form, hiragana, katakana, romaji) #creating the word object
        nodes_list.append(word)
    return nodes_list

def main():
    nodes = populate_nodes(tokens)
    for node in nodes:
        print("word ", node.get_word(), "part of speech ", node.get_pos(), "dictionary form ", node.get_dictionary_form(), "hiragana ", node.get_hira(), "katakana ", node.get_kana())
    #print(create_text(nodes))
    
main()
    

word  武士 part of speech  名詞 dictionary form  武士 hiragana  ぶし katakana  ブシ
word  （ part of speech  補助記号 dictionary form  （ hiragana  （ katakana  （
word  サムライ part of speech  名詞 dictionary form  侍 hiragana  さむらい katakana  サムライ
word  ） part of speech  補助記号 dictionary form  ） hiragana  ） katakana  ）
word  は part of speech  助詞 dictionary form  は hiragana  は katakana  ハ
word  もともと part of speech  副詞 dictionary form  元々 hiragana  もともと katakana  モトモト
word  9 part of speech  名詞 dictionary form  None hiragana  9 katakana  9
word  世紀 part of speech  名詞 dictionary form  世紀 hiragana  せいき katakana  セイキ
word  頃 part of speech  名詞 dictionary form  頃 hiragana  ごろ katakana  ゴロ
word  の part of speech  助詞 dictionary form  の hiragana  の katakana  ノ
word  宮廷 part of speech  名詞 dictionary form  宮廷 hiragana  きゅうてい katakana  キュウテイ
word  社会 part of speech  名詞 dictionary form  社会 hiragana  しゃかい katakana  シャカイ
word  で part of speech  接続詞 dictionary form  で hiragana  で katakana  デ
word  身分 part of speech  名詞 dicti

word  礼儀 part of speech  名詞 dictionary form  礼儀 hiragana  れいぎ katakana  レイギ
word  や part of speech  形状詞 dictionary form  嫌 hiragana  や katakana  ヤ
word  心得 part of speech  名詞 dictionary form  心得 hiragana  こころえ katakana  ココロエ
word  など part of speech  助詞 dictionary form  など hiragana  など katakana  ナド
word  は part of speech  助詞 dictionary form  は hiragana  は katakana  ハ
word  それぞれ part of speech  名詞 dictionary form  其々 hiragana  それぞれ katakana  ソレゾレ
word  の part of speech  助詞 dictionary form  の hiragana  の katakana  ノ
word  武家 part of speech  名詞 dictionary form  武家 hiragana  ぶけ katakana  ブケ
word  で part of speech  接続詞 dictionary form  で hiragana  で katakana  デ
word  解釈 part of speech  名詞 dictionary form  解釈 hiragana  かいしゃく katakana  カイシャク
word  が part of speech  接続詞 dictionary form  が hiragana  が katakana  ガ
word  異なる part of speech  動詞 dictionary form  異なる hiragana  ことなる katakana  コトナル
word  場合 part of speech  名詞 dictionary form  場合 hiragana  ばあい katakana  バアイ
word  も part of speech  助詞 di

word  いう part of speech  動詞 dictionary form  言う hiragana  いう katakana  イウ
word  身分 part of speech  名詞 dictionary form  身分 hiragana  みぶん katakana  ミブン
word  は part of speech  助詞 dictionary form  は hiragana  は katakana  ハ
word  なくなっ part of speech  動詞 dictionary form  無くなる hiragana  なくなっ katakana  ナクナッ
word  て part of speech  接続詞 dictionary form  で hiragana  て katakana  テ
word  も part of speech  助詞 dictionary form  も hiragana  も katakana  モ
word  、 part of speech  補助記号 dictionary form  、 hiragana  、 katakana  、
word  多く part of speech  形容詞 dictionary form  多い hiragana  おおく katakana  オオク
word  の part of speech  助詞 dictionary form  の hiragana  の katakana  ノ
word  元 part of speech  名詞 dictionary form  元 hiragana  もと katakana  モト
word  武士 part of speech  名詞 dictionary form  武士 hiragana  ぶし katakana  ブシ
word  たち part of speech  名詞 dictionary form  質 hiragana  たち katakana  タチ
word  は part of speech  助詞 dictionary form  は hiragana  は katakana  ハ
word  、 part of speech  補助記号 dictionary form  、 h